# Решение практических задач по парсингу данных

## BeautifulSoup4

### Устанавливаем и импортируем необходимые библиотеки

In [ ]:
# Импортируем библиотеки
! pip install fake-useragent
import re
import csv
import time
import requests

from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from tqdm import tqdm
import urllib.parse
from urllib.parse import urljoin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.0 MB/s eta 0:00:00


### Объявляем необходимые функции

In [ ]:
# получение хедерса
def get_headers(ajax=False) -> tuple:
    '''
    Получаем headers для requests

    :param values: - браузеры для ресерча

    Создаем случайного фейк-агента и возвращаем
    '''
    ua = UserAgent()

    headers = {
        'user-agent': ua.random,
        'x-requested-with': 'XMLHttpRequest' if ajax else None
    }

    return headers

# функция для получения ингредиентов супа
def get_response(urln: str, pars='lxml',ajax=False ,enc='utf-8'):
    '''
    Получаем ингредиенты супа

    :param urln: url который добавляем в суп
    :param pars: какой парсинг будем использовать по умолчанию lxml
    :param enc: кодировка для энкодинга

    Получаем реквест по url энкодим по utf-8 по умолчанию
    Получаем ингредиенты и возвращаем
    '''

    response = requests.get(url=urln, headers=get_headers(ajax))
    response.encoding = 'utf-8'

    soup = BeautifulSoup(response.text, pars)
    return soup


# Функция для извлечения чисел из строки
def extract_numbers_from_string(string):
    pattern = r'\d+'  # Регулярное выражение для поиска цифр
    numbers = re.findall(pattern, string)  # Находим все числа в строке
    return numbers


# параметры для ссылки на обменнике крипты
def param_search(name: str, coin: str, count: int, revers=False) -> tuple:
    data = {
        'GiveName': name,
        'GetName': coin,
        'Sum': count,
        'Direction': revers
    }
    return data


# функция для округления float значений до 3-х знаков, даже если там 345.0
def round_float(value: float) -> float:
    '''
    Функия для обрезания дополнительных нулевоезначениов для float

    :param value: float значение

    Создаемая функия обрезает дополнительные нулевоезначения для float
    '''
    if value == 0.0:
        return 0.0

    if value % 1 == 0.0:
        return int(value)

    if value % 1 == 0.5:
        return int(value) + 1

    if value % 1 == 0.25:
        return int(value) + 0.5

    if value % 1 == 0.75:
        return int(value) + 0.5

    if value % 1 == 0.1:
        return int(value) + 0.1
    return round(value, 3)


### Решение задач

---

#### Задача 1

+ Открываем [сайт](http://parsinger.ru/html/index3_page_4.html)
+ Проходимся по всем страницам в категории мыши (всего  4 страницы)
+ На каждой странице посещаем каждую карточку с товаром (всего 32 товаров)
+ В каждой карточке извлекаем при помощи bs4 артикул <p class="article"> Артикул: 80244813 </p>
+ Складываем(плюсуем) все собранные значения
+ Выводим получившийся результат

In [ ]:
for link in soup.find(class_='pagen').find_all('a'):
    url_link = father_url + link['href']
    resp = requests.get(url=url_link, headers=headers)
    resp.encoding = 'utf-8'

    soup = BeautifulSoup(resp.text, 'lxml')

    named.append([name.text for name in soup.find(class_='item_card').find_all('a', class_='name_item')])

In [ ]:
sum(article)

2505109532

---

#### Задача 2

+ Открываем [сайт](http://parsinger.ru/html/index1_page_1.html)
+ Проходимся по всем категориям, страницам и карточкам с товарами(всего 160шт)
+ Собираем с каждой карточки стоимость товара умножая, на количество товара в наличии
+ Складываем получившийся результат
+ Получившуюся цифру с общей стоимостью всех товаров вставляем и выводим.

In [ ]:
url = 'https://parsinger.ru/html/index1_page_1.html'
father_url = 'https://parsinger.ru/html/'

soup = get_response(url)

list_of_expression = []
count = 0
for link in tqdm(soup.find(class_='nav_menu').find_all('a')):
    level1_url = father_url + link['href']
    soup1_level = get_response(level1_url)

    for number in soup1_level.find(class_='pagen').find_all('a'):
        level2_url = father_url + number['href']
        soup2_level = get_response(level2_url)

        for index in soup2_level.find(class_='item_card').find_all('a', class_='name_item'):
            level3_url = father_url + index['href']
            soup3_level = get_response(level3_url)
            price = soup3_level.find('span', id='price').text
            in_stock = soup3_level.find('span', id='in_stock').text
            price_number, in_stock_number = (extract_numbers_from_string(price),
                                             extract_numbers_from_string(in_stock))
            price_number = int(price_number[0])
            in_stock_number = int(in_stock_number[0])
            if in_stock_number == 0:
                list_of_expression.append(price_number * 1)
            else:
                list_of_expression.append(price_number * in_stock_number)

print(f'\n\nСумма всех товаров на сайте: {sum(list_of_expression)}')

100%|██████████| 5/5 [01:15<00:00, 15.14s/it]



Сумма всех товаров на сайте: 45067195


---

#### Задача 3

+ На  [сайте](https://parsinger.ru/table/1/index.html) расположена таблица
+ Цель: собрать все уникальные числа из таблицы (кроме цифр в заголовке) и суммировать их;
+ Полученный результат вывести на экран

In [ ]:
url = 'https://parsinger.ru/table/1/index.html'

soup = get_response(url, 'html.parser')


numberts = []

for number in soup.find_all('td'):
    num = float(number.text)
    if num not in numberts:
        numberts.append(num)
(f'Сумма уникальных цифр в заголовке: {sum(numberts)}')

'Сумма уникальных цифр в заголовке: 1240.0959999999998'

---

#### Задача 4

+ На  [сайте](https://parsinger.ru/table/2/index.html) расположена таблица;
+ Цель: Собрать числа с 1го столбца и суммировать их;
+ Полученный результат вывести на экран.

In [ ]:
url = 'https://parsinger.ru/table/2/index.html'

soup = get_response(url)

first_col = [float(t.text) for t in soup.select('td:first-of-type')]

print(f'Сумма чисел первого столбца: {sum(first_col)}')

Сумма чисел первого столбца: 80.726


---

#### Задача 5

+ На  [сайте](https://parsinger.ru/table/3/index.html) расположена таблица;
+ Цель: Собрать числа которые выделены жирным шрифтом и суммировать их;
+ Полученный результат вывести на экран.

In [ ]:
url = 'https://parsinger.ru/table/3/index.html'
soup = get_response(url)

all_bold_font = [float(b.text) for b in soup.find('table').find_all('b')]

print(f'Сумма чисел выделенных жирным шрифтом: {sum(all_bold_font)}')

Сумма чисел выделенных жирным шрифтом: 373.32899999999995


---

#### Задача 6



+ На  [сайте](https://parsinger.ru/table/4/index.html) расположена таблица;
+ Цель: Собрать числа в зелёных ячейках и суммировать их;
+ Полученный результат вывести на экран.

In [ ]:
url = 'https://parsinger.ru/table/4/index.html'
soup = get_response(url)

all_green = [float(g.text) for g in soup.select('.green')]
print(f'Сумма чисел в зеленых ячейках: {sum(all_green)}')

Сумма чисел в зеленых ячейках: 425.7659999999999


---

#### Задача 7

+ На  [сайте](https://parsinger.ru/table/5/index.html) расположена таблица;
+ Цель: Умножить число в оранжевой ячейке на число в голубой ячейке в той же строке и всё суммировать;
+ Полученный результат вывести на экран

In [ ]:
url = 'https://parsinger.ru/table/5/index.html'
soup = get_response(url)

final_sum = []
for tr in soup.select('tr:has(.orange)'):
    orange = float(tr.select_one('.orange').text)
    blue = float(tr.select_one('td:last-of-type').text)
    final_sum.append(orange * blue)
print(f'Сумма произведения чисел в оранжевой и голубой ячейках: {sum(final_sum)}')

Сумма произведения чисел в оранжевой и голубой ячейках: 2521465.6860000016


---

#### Задача 8

+ На  [сайте](https://parsinger.ru/table/5/index.html) расположена таблица;
+ Цель: Написать скрипт который формирует словарь, где ключ будет автоматически формироваться из заголовка столбцов, а значения это сумма всех чисел в столбце;
+ Округлить каждое число до 3х символов после запятой.
+ Полученный словарь вывести на экран.


Пример ожидаемого словаря:

{'1 column': 000.000, '2 column': 000.000, '3 column': 000.000, '4 column': 000.000, '5 column':
000.00, '6 column': 000.000, '7 column': 000.000, '8 column': 000.000, '9 column': 000.000,
'10 column': 000.000, '11 column': 000.000, '12 column': 000.000, '13 column': 000.000, '14 column':
000.000, '15 column': 000000.0}

In [ ]:
url = 'https://parsinger.ru/table/5/index.html'

soup = get_response(url)

final_tuple = {}

for th in soup.select('th'):
    for td in soup.select('tr:not(:has(th))'):
        list_values = []
        for value in td.text.strip().split('\n'):
            float_value = float(value)
            list_values.append(round(float(value), 3))
        final_tuple[th.text] = sum(list_values)


display(final_tuple)

{'1 column': 1346.644,
 '2 column': 1346.644,
 '3 column': 1346.644,
 '4 column': 1346.644,
 '5 column': 1346.644,
 '6 column': 1346.644,
 '7 column': 1346.644,
 '8 column': 1346.644,
 '9 column': 1346.644,
 '10 column': 1346.644,
 '11 column': 1346.644,
 '12 column': 1346.644,
 '13 column': 1346.644,
 '14 column': 1346.644,
 '15 column': 1346.644}

---

#### Задача 9

Напишите код, который собирает данные в категории [HDD](https://parsinger.ru/html/index1_page_1.html) со всех 4х страниц и сохраняет всё в таблицу.

![](https://ucarecdn.com/5c490d7b-bb04-45c3-a88b-a4bddd482e46/)



Информация которую необходимо собрать.



    Заголовки :  Наименование, Бренд, Форм-фактор, Ёмкость, Объём буф. памяти, Цена

Пример заголовков

![](https://ucarecdn.com/c7589858-d54c-463b-ab65-776eb6225a10/)


In [ ]:
# начало ресерчинга
url = 'https://parsinger.ru/html/index4_page_1.html'

father_url = 'https://parsinger.ru/html/'

# суп начала ресерчинга
soup = get_response(url)

# получаем признаки
items = [item.text.strip() for item in soup.select('.description')[0]]
col_names = ['Наименование'] + [item.split(':')[0].strip()
            for item in items if item
                    .split(':')[0]
                    .strip() != ''] + ['Цена']
# записываем признаки
with open('hdd-list.csv', 'w', encoding='utf-8-sig',  newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(col_names)

# переменная файла для аппенда на итерациях
file = open('hdd-list.csv', 'a', encoding='utf-8-sig', newline='')
writer = csv.writer(file, delimiter=';')

# линки по которым итерируемся
iter_link =[link.get('href') for link in soup.select('div.pagen a')]

# итерация по линкам
for link in tqdm(iter_link):
    # получаем линк страниц на каждой итерации
    url = father_url + link
    # получаем ингредиенты супа
    soup = get_response(url)
    # ждем
    time.sleep(1)
    # получаем наименование
    name = [x.text.strip() for x in soup.select('a.name_item')]
    # описание
    description = [x.text.split('\n') for x in soup.select('div.description')]
    # цену
    price = [x.text for x in soup.select('p.price')]

    # итерируемся по значениям и собираем ингредиенты воедино
    for item, descr, price in zip(name, description, price):
        flatten = item, *[x.split(':')[1].strip()
                        for x in descr if x], price
        # записываем в файл
        writer.writerow(flatten)

# закрывем
file.close()

---

#### Задача 10

Напишите код, который собирает данные со всех страниц и категорий на сайте [тренажере](http://parsinger.ru/html/index1_page_1.html) и сохраните всё в таблицу.

Информация которую необходимо собрать.

![](https://ucarecdn.com/5c490d7b-bb04-45c3-a88b-a4bddd482e46/)

 Заголовки :  Указывать не нужно

Порядок колонок должен быть как на скрине ниже.

![](https://ucarecdn.com/c1fec677-46f3-49c6-99fa-0dbb6c4617f0/)

In [ ]:
# начало ресерчинга
url = 'https://parsinger.ru/html/index4_page_1.html'

general_url = 'https://parsinger.ru/html/'
# суп начала ресерчинга
soup = get_response(url)

# записываем признаки
with open('product-list.csv', 'w', encoding='utf-8-sig',  newline='') as file:
    writer = csv.writer(file, delimiter=';')
# переменная файла для аппенда на итерациях
file = open('product-list.csv', 'a', encoding='utf-8-sig', newline='')
writer = csv.writer(file, delimiter=';')
# линки категорий
link_pages = [link.get('href') for link in soup.select('div.nav_menu a')]
for page in tqdm(link_pages):
# линки категорий по которым итерируемся
    soup = get_response(general_url + page)
    iter_link =[link.get('href') for link in soup.select('div.pagen a')]

    # итерация по линкам страниц
    for link in iter_link:
        # получаем линк страниц на каждой итерации
        url = father_url + link
        # получаем ингредиенты супа
        soup = get_response(url)
        # спим
        time.sleep(1)
        # получаем наименование
        name = [x.text.strip() for x in soup.select('a.name_item')]
        # описание
        description = [x.text.split('\n') for x in soup.select('div.description')]
        # цену
        price = [x.text for x in soup.select('p.price')]

        # итерируемся по значениям и собираем ингредиенты воедино
        for item, descr, price in zip(name, description, price):
            flatten = item, *[x.split(':')[1].strip()
                            for x in descr if x], price
            # записываем в файл
            writer.writerow(flatten)

# закрывем
file.close()

---

#### Задача 11

Напишите код, который собирает данные в категории [watch](http://parsinger.ru/html/index1_page_1.html) c каждой карточки, всего их 32.

**Информация которую необходимо собрать.**

![](https://ucarecdn.com/f29cc133-f193-46dc-8dcc-b2096fd80678/)

Обязательные Заголовки :  Наименование, Артикул, Бренд, Модель, Тип, Технология экрана, Материал корпуса, Материал браслета, Размер, Сайт производителя, Наличие, Цена, Старая цена,  Ссылка на карточку с товаром.

Всего должно быть 14 заголовков

Пример заголовков

![](https://ucarecdn.com/97b2e668-238b-4e14-9ce0-55ff35633fbb/)

p.s. Длинные заголовки можно сократить

In [ ]:
# входной урл
url = 'https://parsinger.ru/html/watch/1/1_2.html'

# первый суп
soup = get_response(url)

# получаем столбцы
art = [p.text.split(':')[0] for p in soup.select('.article')]
col_desc = [de.strip().split(':')[0] for de in soup.find(id='description').text.strip().split('\n')]
last_cols = ['Наличие', 'Цена', 'Старая цена', 'Ссылка на карточку с товаром']

# записываем первую строку признаков
with open('watch-list.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Наименование']+ art + col_desc + last_cols)

# начниаем ресерч
url = 'https://parsinger.ru/html/index1_page_1.html'

father_url = f"{'/'.join(url.split('/')[:-1])}/"
# второй суп
soup = get_response(url)

# функция форматирования текста
def form(tag):
    tag = tag.text.split(':')[-1].strip()
    return tag

# переменная пагинации
pagen = [page.get('href') for page in soup.select_one('div.pagen').find_all('a')]

# списки объектов
named = []
articl = []
descriptions = []
in_stocks = []
prices = []
old_price = []
product_url = []

# пагинация
for page in tqdm(pagen):
    # получаем первый урл и суп
    url = urljoin(father_url, page)
    soup = get_response(url)

    # пошли в карточки
    product_card = [prod.get('href') for prod in soup.select('.sale_button a')]
    for product in product_card:
        # второй уровень - урл, суп
        url = urljoin(father_url, product)
        soup = get_response(url)

        # получаем все необходимые данные
        names = [name.text.strip() for name in soup.select('#p_header')]
        article = [form(art) for art in soup.select('.article')]
        description = [form(des) for des in soup.find(id='description').find_all('li')]
        in_stock = [form(stock) for stock in soup.select('#in_stock')]
        price = [old_p.text for old_p in soup.select('span[id$="price"]')]

        # добавляем в списки объекты
        named.append(names)
        articl.append(article)
        descriptions.append(description)
        in_stocks.append(in_stock)
        prices.append(price)
        product_url.append(url)

# итерируемся и записываем
for name, art, descr, stock, price, link in zip(named, articl,
                                                descriptions, in_stocks,
                                                prices, product_url):
    row = *name, *art, *descr, *stock, *price, link
    file = open('watch-list.csv', 'a', encoding='utf-8-sig', newline='')
    writer = csv.writer(file, delimiter=';')
    writer.writerow(row)

# закрываем и выводим ответ
file.close()
print('Файл создан')

---

#### Задача 12

Напишите код, который собирает данные в каждой категории c каждой карточки, всего их 160.

Информация которую необходимо собрать.

![](https://ucarecdn.com/49455ee6-6810-47af-a4d8-b8f6485b4d70/)

Обязательные Заголовки :  Наименование, Артикул, Бренд, Модель, Наличие, Цена, Старая цена, Ссылка на карточку с товаром,

Перечисленные заголовки являются общими для всех карточек.

Пример заголовков.

 ![](https://ucarecdn.com/04a8d7d1-60cb-47c0-834e-7ce3016b91fa/)







In [ ]:
# входной урл
url = 'https://parsinger.ru/html/watch/1/1_2.html'

# первый суп
soup = get_response(url)

# получаем столбцы
art = [p.text.split(':')[0] for p in soup.select('.article')]
col_desc = [de.strip().split(':')[0] for de in soup.find(id='description').text.strip().split('\n')]
last_cols = ['Наличие', 'Цена', 'Старая цена', 'Ссылка на карточку с товаром']

# записываем первую строку признаков
with open('prod-list.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Наименование']+ art + col_desc[:2] + last_cols)

# начниаем ресерч
url = 'https://parsinger.ru/html/index1_page_1.html'

father_url = f"{'/'.join(url.split('/')[:-1])}/"
# второй суп
soup = get_response(url)

# функция форматирования текста
def form(tag):
    tag = tag.text.split(':')[-1].strip()
    return tag

# списки объектов
named = []
articl = []
descriptions = []
in_stocks = []
prices = []
old_price = []
product_url = []

# переменная каталога
catalog_pages = [a['href'] for a in soup.find('div', 'nav_menu').find_all('a')]

# пагинация по каталогу
for cat in tqdm(catalog_pages):
    url = urljoin(father_url, cat)
    soup = get_response(url)

    # переменная пагинации
    pagen = [page.get('href') for page in soup.select_one('div.pagen').find_all('a')]
    for page in pagen:
        # получаем второй урл и суп
        url = urljoin(father_url, page)
        soup = get_response(url)

        # пошли в карточки
        product_card = [prod.get('href') for prod in soup.select('.sale_button a')]
        for product in product_card:
            # третий уровень - урл, суп
            url = urljoin(father_url, product)
            soup = get_response(url)

            # получаем все необходимые данные
            names = [name.text.strip() for name in soup.select('#p_header')]
            article = [form(art) for art in soup.select('.article')]
            description = [form(des) for des in soup.find(id='description').find_all('li')]
            in_stock = [form(stock) for stock in soup.select('#in_stock')]
            price = [old_p.text for old_p in soup.select('span[id$="price"]')]

            # добавляем в списки объекты
            named.append(names)
            articl.append(article)
            descriptions.append(description[:2])
            in_stocks.append(in_stock)
            prices.append(price)
            product_url.append(url)

# итерируемся и записываем
for name, art, descr, stock, price, link in zip(named, articl,
                                                descriptions, in_stocks,
                                                prices, product_url):
    row = *name, *art, *descr, *stock, *price, link
    file = open('prod-list.csv', 'a', encoding='utf-8-sig', newline='')
    writer = csv.writer(file, delimiter=';')
    writer.writerow(row)

# закрываем и выводим ответ
file.close()
print('Файл создан')

---


#### Задача 13

Выберите 1 любую категорию на сайте [тренажере](http://parsinger.ru/html/index1_page_1.html) и соберите все данные с карточек.

![](https://ucarecdn.com/403eeeac-1048-4bdf-9689-d5b5016bb40c/)

По результату выполнения кода должен появится файл .json с отступом в 4 пробела




In [ ]:
url = 'https://parsinger.ru/html/index2_page_1.html'

soup = get_response(url)

keys = (
    ['Наименование'] + [li.text.split(':')[0] for li in soup
                                .select('.description')[0]
                                .find_all('li')] + ['Цена'])

res = []
names = [n.text for n in soup.select('.name_item')]
description = [x.text.strip().split('\n') for x in soup.select('div.description')]
prices = [d.text for d in soup.select('p.price')]

for name, desc, price in zip(names, description, prices):
    res.append({
        keys[0]: name,
        keys[1]: desc[0].split(':')[1],
        keys[2]: desc[1].split(':')[1],
        keys[3]: desc[2].split(':')[1],
        keys[4]: desc[3].split(':')[1],
        keys[5]: price
    })

with open('res.json', 'w', encoding='utf-8') as file:
    json.dump(res, file, indent=4, ensure_ascii=False)

---

#### Задача 14

Соберите данные со всех 5 категорий на сайте [тренажере](http://parsinger.ru/html/index1_page_1.html) и соберите все данные с карточек.


![](https://ucarecdn.com/f958fc45-0139-4c56-9539-189d1615f6cb/)

По результату выполнения кода должен появится файл .json с отступом в 4 пробела

Пример:

![](https://ucarecdn.com/9088b7b7-7041-41ff-94c2-ce6dbf7ecee3/)




In [ ]:
# стартовый урл
url = 'https://parsinger.ru/html/index1_page_1.html'

res1 = []

soup = get_response(url)
father_url = f"{'/'.join(url.split('/')[:-1])}/"
# переменная каталога
catalog_pages = [a['href'] for a in soup.find('div', 'nav_menu').find_all('a')]

# первый уровень
for cat in tqdm(catalog_pages):
    url = urljoin(father_url, cat)
    soup = get_response(url)
      # переменная пагинации
    pagen = [page.get('href') for page in soup.select_one('div.pagen').find_all('a')]

    # второй уровень
    for page in pagen:
        # получаем второй урл и суп
        url = urljoin(father_url, page)
        soup = get_response(url)

        keys = (
            ['Наименование'] + [li.text.split(': ')[0] for li in soup
                                        .select('.description')[0]
                                        .find_all('li')] + ['Цена'])


        names = [n.text for n in soup.select('.name_item')]
        description = [x.text.strip().split('\n') for x in soup.select('div.description')]
        prices = [d.text for d in soup.select('p.price')]

        for name, desc, price in zip(names, description, prices):
            res1.append({
                keys[0]: name,
                keys[1]: desc[0].split(':')[1].strip(),
                keys[2]: desc[1].split(':')[1].strip(),
                keys[3]: desc[2].split(':')[1].strip(),
                keys[4]: desc[3].split(':')[1].strip(),
                keys[5]: price
            })

with open('res1.json', 'w', encoding='utf-8') as file:
    json.dump(res1, file, indent=4, ensure_ascii=False)

---

#### Задача 15

Выберите 1 любую категорию на сайте [тренажёре](http://parsinger.ru/html/index3_page_1.html), и соберите все данные с карточек товаров + ссылка на карточку.

![](https://ucarecdn.com/f84ae3da-5341-4295-ab9b-13c35599234c/)

По результату выполнения кода должен появится файл .json с отступом в 4 пробела. Ключи в блоке description должны быть получены автоматически из атрибутов HTML элементов.



Пример:

![](https://ucarecdn.com/5f6db4be-c93c-4851-a99d-f1c31e49cfbf/)

In [ ]:
# стартовый урл
url = 'https://parsinger.ru/html/watch/1/1_2.html'

# первый суп
soup = get_response(url)

# получаем столбцы
art = [p.text.split(':')[0] for p in soup.select('.article')]
col_desc = [de.strip().split(':')[0] for de in soup.find(id='description').text.strip().split('\n')]
last_cols = ['Наличие', 'Цена', 'Старая цена', 'Ссылка на карточку с товаром']

# записываем первую строку признаков
with open('prod-list.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Наименование']+ art + col_desc[:2] + last_cols)

# начниаем ресерч
url = 'https://parsinger.ru/html/index1_page_1.html'

father_url = f"{'/'.join(url.split('/')[:-1])}/"
# второй суп
soup = get_response(url)

# функция форматирования текста
def form(tag):
    tag = tag.text.split(':')[-1].strip()
    return tag

# списки объектов
result = []


# переменная пагинации
pagen = [page.get('href') for page in soup.select_one('div.pagen').find_all('a')]
for page in tqdm(pagen):
    # получаем второй урл и суп
    url = urljoin(father_url, page)
    soup = get_response(url)

    # переходим в карточки
    product_card = [prod.get('href') for prod in soup.select('.sale_button a')]
    for product in product_card:
        # третий уровень - урл, суп
        url = urljoin(father_url, product)
        soup = get_response(url)

        art = [p.text.split(':')[0] for p in soup.select('.article')]
        col_desc = [de.strip().split(':')[0] for de in soup.find(id='description').text.strip().split('\n')]
        last_cols = ['Наличие', 'Цена', 'Старая цена', 'Ссылка на карточку с товаром']

        keys = (['Наименование']+ art + col_desc + last_cols)

        # получаем все необходимые данные
        names = [name.text.strip() for name in soup.select('#p_header')]
        article = [form(art) for art in soup.select('.article')]
        description = [form(des) for des in soup.find(id='description').find_all('li')]
        in_stock = [form(stock) for stock in soup.select('#in_stock')]
        price = [old_p.text for old_p in soup.select('span[id$="price"]')]

        # добавляем объекты
        for name, art, stock in zip(names, article, in_stock):
            result.append({
                keys[0]: name,
                keys[1]: art,
                keys[2]: description[0],
                keys[3]: description[1],
                keys[4]: description[2],
                keys[5]: description[3],
                keys[6]: description[4],
                keys[7]: description[5],
                keys[8]: description[6],
                keys[9]: description[7],
                keys[10]: stock,
                keys[11]: prices[0],
                keys[12]: prices[1],
                keys[13]: url,
            })

with open('result1.json', 'w', encoding='utf-8') as file:
    json.dump(result, file, indent=4, ensure_ascii=False)

print('Файл создан')

---

#### Задача 16

Соберите данные со всех 5 категорий на сайте [тренажере](http://parsinger.ru/html/index1_page_1.html) и соберите все данные с карточек + ссылка на карточку с товаром.

![](https://ucarecdn.com/5fe217bc-6960-4220-8286-caa2c910ca84/)

По результату выполнения кода должен появится файл .json с отступом в 4 пробела. Ключи в блоке description должны быть получены автоматически из атрибутов HTML элементов.



Пример:

![](https://ucarecdn.com/0c1bb5e4-0273-4c9a-8335-c0476497a23d/)

In [ ]:
# стартовый урл
url = 'https://parsinger.ru/html/watch/1/1_2.html'

# первый суп
soup = get_response(url)

# получаем столбцы
art = [p.text.split(':')[0] for p in soup.select('.article')]
col_desc = [de.strip().split(':')[0] for de in soup.find(id='description').text.strip().split('\n')]
last_cols = ['Наличие', 'Цена', 'Старая цена', 'Ссылка на карточку с товаром']

# записываем первую строку признаков
with open('prod-list.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Наименование']+ art + col_desc[:2] + last_cols)


url = 'https://parsinger.ru/html/index1_page_1.html'

father_url = f"{'/'.join(url.split('/')[:-1])}/"
# второй суп
soup = get_response(url)

# функция форматирования текста
def form(tag):
    tag = tag.text.split(':')[-1].strip()
    return tag

# списки объектов
result = []

# переменная каталога
catalog_pages = [a['href'] for a in soup.find('div', 'nav_menu').find_all('a')]

for cat in tqdm(catalog_pages):
    url = urljoin(father_url, cat)
    soup = get_response(url)

    # переменная пагинации
    pagen = [page.get('href') for page in soup.select_one('div.pagen').find_all('a')]
    for page in pagen:
        # получаем второй урл и суп
        url = urljoin(father_url, page)
        soup = get_response(url)

        # пошли в карточки
        product_card = [prod.get('href') for prod in soup.select('.sale_button a')]
        for product in product_card:
            # третий уровень - урл, суп
            url = urljoin(father_url, product)
            soup = get_response(url)

            art = [p.text.split(':')[0] for p in soup.select('.article')]
            col_desc = [de.strip().split(':')[0] for de in    soup.find(id='description').text.strip().split('\n')]
            last_cols = ['Наличие', 'Цена', 'Старая цена', 'Ссылка на карточку с товаром']

            keys = (['Наименование']+ art + col_desc + last_cols)

            # получаем все необходимые данные
            names = [name.text.strip() for name in soup.select('#p_header')]
            article = [form(art) for art in soup.select('.article')]
            description = [form(des) for des in soup.find(id='description').find_all('li')]
            in_stock = [form(stock) for stock in soup.select('#in_stock')]
            price = [old_p.text for old_p in soup.select('span[id$="price"]')]

            # добавляем объекты
            for name, art, stock in zip(names, article, in_stock):
                result.append({
                    keys[0]: name,
                    keys[1]: art,
                    keys[2]: description[0],
                    keys[3]: description[1],
                    keys[4]: description[2],
                    keys[5]: description[3],
                    keys[6]: description[4],
                    keys[7]: description[5],
                    keys[8]: description[6],
                    keys[9]: description[7],
                    keys[10]: stock,
                    keys[11]: prices[0],
                    keys[12]: prices[1],
                    keys[13]: url,
                })

with open('result.json', 'w', encoding='utf-8') as file:
    json.dump(result, file, indent=4, ensure_ascii=False)

print('Файл создан')

---

#### Задача 17

Используйте полученный по [ссылке](http://parsinger.ru/downloads/get_json/res.json) JSON, чтобы посчитать количество товара в каждой категории.

На вход ожидается словарь {'watch': N, 'mobile': N, 'mouse': N, 'hdd': N, 'headphones': N}, где N - это общее количество товаров

Количество вы найдёте в каждой карточке товара.

In [ ]:
url ='https://parsinger.ru/downloads/get_json/res.json'

response = requests.get(url=url).json()

keys = list(set([item['categories'] for item in response]))[::-1]

category_counts = {key: 0 for key in keys}

for item in response:
    category = item['categories']
    if category in category_counts:
        category_counts[category] += int(item['count'])

print(category_counts)

{'mouse': 2692, 'mobile': 820, 'watch': 853, 'headphones': 1006, 'hdd': 1273}


---

### Практика парсинга курса криптовалюты

Запрашиваем курс криптовалюты на ajax

In [ ]:
# Тестирование запросов перед созданием универсального интерфейса
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest'
}

url = "https://bitality.cc/Home/GetSum?GiveName=Tinkoff&GetName=Bitcoin&Sum=1&Direction=1"
response = requests.get(url=url, headers=headers).json()
print(response)

In [ ]:
# Тестирование с интерфейсом в лице функции
url = 'https://bitality.cc/Home/GetSum' # с вопросом в конце и без работает, реквест подставляет сам вопрос

data = param_search('Sberbank', 'Bitcoin', 5_000)

headers = get_headers(1)

response = requests.get(url=url,headers=headers ,params=data).json()
print(response)

{'giveSum': 5000.0, 'getSum': '0.00194801'}
